In [1]:
import logging
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
logging.getLogger().setLevel(logging.ERROR)
import tensorflow as tf
tf.get_logger().setLevel(logging.ERROR)

import sys
sys.path.append("../src")

from Evaluation import run_ensembles
from LoadData import CurrentDatasets
from Helpers import create_confusion_matrix_plot_from_csv

In [2]:
dataset_names = [dataset.value for dataset in CurrentDatasets]

ensembles = {"ALL": ["Encoder", "FCN", "MCDCNN", "MLP", "Resnet", "Time_CNN"],
             "BEST3": ["Encoder", "Resnet", "MLP"]}
csv_name = '../results/test_Ensembles.csv'

In [ ]:
run_ensembles(dataset_names=dataset_names, ensembles=ensembles, verbose=True).to_csv(csv_name)

1/11:	cricket_y


/home/felix/miniconda3/envs/tf/lib/python3.10/site-packages/keras/layers/core/lambda_layer.py:324: UserWarning: ModelBuilder is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(


13/13 [==============================] - 0s 1ms/step


/home/felix/miniconda3/envs/tf/lib/python3.10/site-packages/keras/layers/core/lambda_layer.py:324: UserWarning: ModelBuilder is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(


13/13 [==============================] - 0s 5ms/step
2/11:	distal_phalanax_tw


...

1/1 [==============================] - 0s 13ms/step
4/11:	electric_devices


/home/felix/miniconda3/envs/tf/lib/python3.10/site-packages/keras/layers/core/lambda_layer.py:324: UserWarning: ModelBuilder is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(


241/241 [==============================] - 0s 1ms/step


/home/felix/miniconda3/envs/tf/lib/python3.10/site-packages/keras/layers/core/lambda_layer.py:324: UserWarning: ModelBuilder is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(


  1/241 [..............................] - ETA: 2s

In [ ]:
create_confusion_matrix_plot_from_csv(csv_name, verbose=True)